# NeuroEvolution of Augmenting Topologies (NEAT)
Evolving the weights and the topologies of a Neural Networks using the NEAT algorithm.

In [1]:
import importlib
import environment 
importlib.reload(environment)

import numpy as np
import neat

In [2]:
initial_setting = {
    'agents': np.array([[0, 5], [0, 10], [0, 15]], dtype=float),
    'blocks': np.array([[9, 16], [13, 7], [6, 5], [10, 11]], dtype=float),
    'colors': np.array([environment.RED, environment.RED, environment.BLUE, environment.GREEN], dtype=int)
}
env = environment.Environment(objective = [(environment.RED, environment.NORTH_EDGE)],
                   size = environment.SIMULATION_ARENA_SIZE, 
                   n_agents = 3, 
                   n_blocks = 4,
                   n_neighbors = 3,
                   sensor_range = environment.SIMULATION_SENSOR_RANGE,
                   sensor_angle = 360,
                   max_distance_covered_per_step = environment.SIMULATION_MAX_DISTANCE,
                   sensitivity = 0.5,
                   initial_setting = initial_setting)
env.reset()
env.print_env()

. . . . . . 0 . . . . . . 1 . . . . . 2 . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . O . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . O . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . O . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . O . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .


In [3]:
def calculate_fitnesses(genomes, config, n_steps = 100, verbose=False):
    
    for genome_id, genome in genomes:
        genome.fitness = 0.0
        
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        obs, _ = env.reset()
        if verbose: env.print_env()

        for step in range(n_steps):
            
            nn_inputs = env.process_observation(obs)
            
            nn_outputs = [net.activate(nn_input) for nn_input in nn_inputs]

            actions = np.round(nn_outputs * np.array([env.max_distance_covered_per_step, env.sensor_angle]), 1)
            
            obs, reward, done, _, _ = env.step(actions)
            genome.fitness += reward

            if verbose:
                print("\nStep", i)
                # print("Observation: ", obs)
                # print("NN inputs: ", nn_inputs)
                print("Action: ", actions)
                env.print_env()
                print("Reward: ", reward)
            
            if done:
                genome.fitness += (n_steps - step) / 2
                break

In [4]:
# Set configuration file
config_path = "./neat_config.txt"
config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                            neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

# Create core evolution algorithm class
p = neat.Population(config)

# Add reporter for fancy statistical result
p.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
p.add_reporter(stats)

# Run NEAT
winner = p.run(calculate_fitnesses, 100)


 ****** Running generation 0 ****** 

Population's average fitness: -116.55795 stdev: 250.67826
Best fitness: 1.34701 - size: (2, 88) - species 1 - id 17
Average adjusted fitness: 0.856
Mean genetic distance 1.264, standard deviation 0.378
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   100      1.3    0.856     0
Total extinctions: 0
Generation time: 8.397 sec

 ****** Running generation 1 ****** 

Population's average fitness: -59.66892 stdev: 184.78824
Best fitness: 1.34808 - size: (2, 87) - species 1 - id 163
Average adjusted fitness: 0.912
Mean genetic distance 1.267, standard deviation 0.383
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   100      1.3    0.912     0
Total extinctions: 0
Generation time: 6.479 sec (7.438 average)

 ****** Running generation 2 ****** 

Population's average fitness: -106.

Visualize the behaviour of the best individual

In [5]:
calculate_fitnesses([(1, winner)], config, verbose=True)

. . . . . . 0 . . . . . . 1 . . . . . 2 . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . O . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . O . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . O . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . O . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
